In [2]:
import tensorflow as tf

import time

Prefetch

In [4]:
class FileDataset(tf.data.Dataset):
    def read_files_in_batches(num_samples):
        ## Open file
        time.sleep(0.03) ## Open file
        for sample_idx in range(num_samples):
            time.sleep(0.015) ## Read row
            yield(sample_idx, )
    
    def __new__(self, num_samples=3):
        return tf.data.Dataset.from_generator(
            self.read_files_in_batches,
            output_signature=tf.TensorSpec(shape=(1,), dtype= tf.int64),
            args=(num_samples, )
        )

In [5]:
def benchmark(dataset, num_epochs=2):
    for epoch in range(num_epochs):
        for sample in dataset:
            time.sleep(0.01) ## Training

In [13]:
## I don't have any GPU because of that prefetch can't improve benchmark

In [10]:
%%timeit
benchmark(FileDataset())

336 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

351 ms ± 28.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Cache

In [14]:
def mapped_function(s):
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [ ]:
## Cache provides us less time to prepare data to model.
 # when first epoch manupilate data to model and then others epoch use the cached data to feed model

In [15]:
%%timeit -n1 -r1

benchmark(FileDataset().map(mapped_function), 5)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [16]:
%%timeit -n1 -r1

benchmark(FileDataset().map(mapped_function).cache(), 5)

585 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
